In [ ]:
# importing dependencies
import pandas as pd
from sqlalchemy import create_engine
# need this later when creating sql database
import config

In [ ]:
# read in the dataset and put into a dataframe
chicago_crime_file = "data/chicago_crime_stats.csv"
chicago_crime_file

In [ ]:
chicago_crime_df = pd.read_csv(chicago_crime_file)
# looking at the dataframe
chicago_crime_df

# Transform
Here we take our dataframe and drop the columns we do not want included. 

In [ ]:
# columns we want to keep and include in the new 
chicago_crime_cols = ["ID", "Date", "Primary Type", "Description", "Arrest", "Domestic", "District", "Year", "Latitude",
                     "Longitude"]

# creating a new dataframe of the columns we want to include in our database
chicago_crime_stats_df = chicago_crime_df[chicago_crime_cols].copy()

In [ ]:
# looking at our new dataframe
chicago_crime_stats_df = chicago_crime_stats_df.rename(columns=str.lower)

chicago_crime_stats_df.head()

# Limit the dataframe to violent crimes only

In [ ]:
# take only those Primary Type's that are considered violent crimes

In [ ]:
# checking the length of the dataframe
len(chicago_crime_stats_df)

In [ ]:
# limit dataframe to homicide, battery, assault, robbery, and sex offense
violent_crimes_df = chicago_crime_stats_df.loc[(chicago_crime_stats_df["primary type"]=="HOMICIDE") | 
                                               (chicago_crime_stats_df["primary type"]=="BATTERY") |
                                               (chicago_crime_stats_df["primary type"]=="ASSAULT") |
                                               (chicago_crime_stats_df["primary type"]=="ROBBERY") |
                                               (chicago_crime_stats_df["primary type"]=="SEX OFFENSE")]

# rename primary type as Primary_Type
violent_crimes_df = violent_crimes_df.rename(columns={"primary type": "primary_type"})

# looking at violent crimes only
violent_crimes_df.head()

In [ ]:
# format the date in the first column using datetime
# first need to take the time out of the row
violent_crimes_df["date"] = violent_crimes_df["date"].str.split(" ", 1, expand=True)

# now format the date using datetime
violent_crimes_df["date"] = pd.to_datetime(violent_crimes_df["date"],format="%m/%d/%y")

violent_crimes_df.head()

In [ ]:
# set date as the index for SQL database
violent_crimes_df.set_index("id", inplace=True)
violent_crimes_df.head()

In [ ]:
# checking the length of the new dataframe
len(violent_crimes_df)

In [ ]:
violent_crimes_df= violent_crimes_df.head(10000)
len(violent_crimes_df)

In [ ]:
#list unique values in the df ['primary_type']
violent_crimes_df.primary_type.unique()

In [ ]:
# checking the data types
print(violent_crimes_df.dtypes)

# Load the data into SQL Database

In [ ]:
# Create a SQL Database connection
connection_string = f"postgres:{config.password}@localhost:5432/chicago_violent_crimes_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# checking tables
engine.table_names()

In [ ]:
# load dataframes into database
try:
    violent_crimes_df.to_sql(name="violent_crimes", con=engine, if_exists="append", index=True)
    print("Data loaded successfully")
    
except:
    print("Data has already been loaded to db")